This technique is taken from here: [Webscraping Multiple
Pages](https://rstudio-pubs-static.s3.amazonaws.com/287509_dde580b0adf94bae8a7994d7ae0cb849.html)

This related page is also a nice example of webscraping prices: [ML
Books - Price Scraping](http://rpubs.com/Marcelobn/ML_books)

Prices, discounts, and names of products are very interesting things to
scrape from websites. That is what we want to do here. Often times there
are page after page of products so we would like to find a way to scrape
multiple pages of the same kind of data.

Install rvest if needed and then load it:

In [ ]:
if (!require("rvest")) install.packages("rvest")
library("rvest")

Now first we just pull one page of data like before. And look for the
prices. We will get a page off of \[Rent the Runway\]
(<a href="https://www.renttherunway.com" class="uri">https://www.renttherunway.com</a>),
we are interested in comparing the retail price for something and the
“one-time” rental that renttherunway charges for those not signed up for
their service. You can use the Selector Gadget in chrome to see what css
selectors you need to grab the retail price and the range that the piece
would rent for. Here the retail price is identified by the selector
`.product-card-price__retail-quiet`, and the range of rental prices is
in the selector `.product-card-price__regular`

In [ ]:
library("rvest")
url <- "https://www.renttherunway.com/c/dresses?page=1"
webpage <- read_html(url)
nodes <- html_nodes(webpage,'.product-card-price__retail-quiet')
retail <- html_text(nodes)
retail

We can see we need to get rid of the `retail` and later on the \$. We
leave it in for the moment:

In [ ]:
retail<-gsub(" retail","", retail, fixed=TRUE)
retail

That looks pretty good now. Next lets work on rental price. This is
given as a range of values on the website, so we will need to pull that
range, and then deal with the high and low values of the range of the
rental price:

In [ ]:
nodes <- html_nodes(webpage,'.product-card-price__regular')
rental <- html_text(nodes)
rental

We have to remove this funny dash in the string. Then we use something
that just extracts the low price. That is what we will work with for
now. We could take an average of the high and low values with a little
more work, but okay:

In [ ]:
rental <- strsplit(rental, " – ",)
lower <- sapply(rental, function(x) x[1])
upper <- sapply(rental, function(x) x[2])
lower
upper

Now lets see if we can pull multiple pages for of this data.

First lets take a look at the following construct:

In [ ]:
pageUrls<-paste0('https://www.renttherunway.com/c/dresses?page=', 1:3)
pageUrls

What this does is just make a vector of length 3 with the urls of the
first three pages that we want to pull.

Now for this vector of urls, lets actually read the html from each of
the pages, and store that in the webpages vector. Then we can use that
to process each field we want from the different pages.

In [ ]:
webpages <- lapply(pageUrls, read_html)

Now there is a function called `lapply` that lets you pass in a list or
vector and it will apply a function you provide to this list and return
the results of that. In this case we pass in a function that does
exactly what we did above when we read the url (a page that is) then
found the retail and rental ranges

Here are the retail prices:

In [ ]:
retail <- lapply(webpages,
        function(webpage){
        nodes<-html_nodes(webpage, '.product-card-price__retail-quiet')
        price <- html_text(nodes) 
        gsub(" retail","", price)
        })
retail <- unlist(retail)
retail

Here is are the rental amounts (the lower of the range):

In [ ]:
rental <- lapply(webpages,
        function(webpage){
        nodes <- html_nodes(webpage,'.product-card-price__regular')
        range <- html_text(nodes)
        range <- strsplit(range, " – ",)
        sapply(range, function(x) x[1])
        })
rental <- unlist(rental)

Finally lets clean up the dollar sign and the comma if numbers are over
1,000:

In [ ]:
retail <-gsub(",","", retail)
rental <-gsub(",","", rental)

retail <-gsub("$","", retail, fixed=TRUE)
rental <-gsub("$","", rental, fixed=TRUE)

Now we should be all set, lets turn them into numbers!

In [ ]:
retail <- as.numeric(retail)
rental <- as.numeric(rental)

In [ ]:
df = data.frame(retail, rental)
df

Just for fun, do you think we can predict the rental price of something
if we know the retail price?

In [ ]:
plot(retail, rental, pch=20)

Looks pretty good…

In [ ]:
options("scipen"=100, "digits"=4)
model<-lm(rental ~ retail, data=df)
summary(model)

Nice… What about what percent the rental is of the retail:

In [ ]:
percentage <- rental/retail
percentage

Whats the range you can expect?

In [ ]:
descriptive<-summary(percentage)
descriptive

Looks like….

**For one time rental only…The runway can be yours for from NA% to NA%
of the retail price!!!!**

Nice!